# 1. Import the libs and base ontologies
one time code to import EMMO.

In [1]:
from owlready2 import *
import pandas as pd
from neo4j import GraphDatabase, basic_auth
from owlready2 import get_ontology
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer, util
import torch
import re
import random
from datetime import datetime



onto_path.append("")
onto_process=get_ontology("C://Abhishek Thesis//ontology//manufacturing.owl").load()

onto_matter=get_ontology("https://raw.githubusercontent.com/IEK-13/MatGraphAI/master/Ontology/matter.owl").load()

onto_quantity=get_ontology("C://Abhishek Thesis//ontology//quantities.owl").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# results = onto_quantity.search_one(label="properties.Quantity")
# print(results)

# 2. Creating node-class 'EMMO_IS_A' relations between exisiting classes.
One time code to extract MATTER, PROCESS and QUANTITY. 

## map the 3 ontologies on neo4j

In [3]:



# Define Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

emmo_mat="EMMOMatter"
emmo_proc="EMMOProcess"
emmo_quant="EMMOQuantity"

# Create Neo4j driver and session
driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

# Define function to create nodes
def create_node(tx, label, name,iri):
    tx.run("MERGE (n:" + label + " {name: $name, uri:$uri})", name=name, uri=iri)
    

# Define function to create EMMO_IS_A relationships
def create_emmo_is_a_rel(tx,class_, subclass, superclass):
    tx.run("MATCH (c1:" + class_ + " {name: $subclass}), (c2:" + class_ + " {name: $superclass}) "
           "MERGE (c1)-[:EMMO_IS_A]->(c2)",class_=class_ ,subclass=subclass, superclass=superclass)

    
    
    
# Create nodes for each class in Matter ontology
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        create_node(tx, emmo_mat, clazz.name,clazz.iri)

# Create nodes for each class in Process ontology
with session.begin_transaction() as tx:
    for clazz in onto_process.classes():
        create_node(tx, emmo_proc, clazz.name,clazz.iri)

# Create nodes for each class in Quantity ontology
with session.begin_transaction() as tx:
    for clazz in onto_quantity.classes():
        create_node(tx, emmo_quant, clazz.name,clazz.iri)

# Create EMMO_IS_A relationships for each class
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,emmo_mat, clazz.name, parent.name)
    for clazz in onto_process.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,emmo_proc, clazz.name, parent.name)
    for clazz in onto_quantity.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,emmo_quant, clazz.name, parent.name)

# Close Neo4j session and driver
session.close()
driver.close()


In [4]:
#To extend the imaging techniques in EMMOProcess-cretae subclasses like FIBImaging,AFMImaging... 
## ONE TIME CODE.


def create_subclass(driver, node_label, superclass_name, subclass_name,relationship_name, url):
    with driver.session() as session:
        result = session.run(
            "MATCH (superclass:" + node_label + ") "
            "WHERE superclass.name = $superclassName "
            "MERGE (subclass:" + node_label + " { name: $subclassName }) "
            "ON CREATE SET subclass.url = $url "
            "MERGE (subclass)-[:"+relationship_name+"]->(superclass) "
            "RETURN subclass, superclass",
            superclassName=superclass_name,
            subclassName=subclass_name,
            url=url + subclass_name  # Append subclass_name to the URL
        )

with driver.session() as session:
    with session.begin_transaction() as tx:
        node_label = emmo_proc
        url_manuf = 'http://www.example.com/manufacturing#'
        relation_emmo_is_a='EMMO_IS_A'
        combinations = [
            ['ElectronMicroscopy', 'FIBImaging'],
            ['ElectronMicroscopy', 'IC_SEMImaging'],
            ['Microscopy', 'ScanningProbeMicroscopy'],
            ['Imaging', 'Neutron'],
            ['ScanningProbeMicroscopy', 'AFMImaging'],
            ['Imaging', 'XRay'],
            ['XRay', 'Synchrotron'],
            ['Synchrotron', 'SynchrotronRadiography'],
            ['Synchrotron', 'SynchrotronTomography']
        ]
        
        for combination in combinations:
            superclass_name = combination[0]
            subclass_name = combination[1]
            
            create_subclass(driver, node_label, superclass_name, subclass_name, relation_emmo_is_a,url_manuf)

# 3. Get the Organizational Data from file

In [5]:
name_property='name'
is_a_rel='IS_A'
emmo_mat="EMMOMatter"
emmo_proc="EMMOProcess"
emmo_quant="EMMOQuantity"



# Read the Excel file
excel_file = 'C://Abhishek Thesis//OneDrive_1_5-11-2023//test file-DataCatalog//DataCatalog.xlsx'
df = pd.read_excel(excel_file, header=None)

# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Organizational Data'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Characterization Data'].index[0]

# Extract the organizational data rows
organizational_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
organizational_data.loc[:, 0] = organizational_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = organizational_data.iloc[:, 0].tolist()
values = organizational_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
data_dict = dict(zip(attributes, values))

# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    globals()[var_name] = value

# Print the variables to verify they contain the expected values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    print(f"{var_name}: {globals()[var_name]}")


experiment_title: Elucidating the Influence of the d-Band Center on the Synthesis of Isobutanol
experimentid: 000000-1
measurementid: 000000-1-2
upload_date: 2022-10-11 00:00:00
measurement_date: 2022-10-12 00:00:00
institution: FZJ IEK-13,FZJ IEK-14
founding_body: HIP, ABBC
country: USA,Japan
authors_list: Peter, Alex
orcid: HJGDF78,uibu5
access_conditions: public
published: https://doi.org/10.3390/catal11030406
topic: Fuel Cells
device: FuelCell
component: WorkingElectrode
subcomponent: CatalystLayer
material: LiquidElectrolyte
granularity_level: Nanostructure


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_13476\1028601918.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  organizational_data.loc[:, 0] = organizational_data.loc[:, 0].str.strip()


# 4. Load this file data to the neo4j db and make relationships

In [6]:
   
    
def create_nodes(tx, label, **properties):
    query = f"CREATE (n:{label}) SET n += $properties RETURN n"
    return tx.run(query, properties=properties)

def create_or_get_node(tx, label, match_property, properties):
    query = f"MERGE (n:{label} {{{match_property}: $value}}) SET n += $properties RETURN n"
    result = tx.run(query, value=properties[match_property], properties=properties)
    if not result.peek():
        create_nodes(tx, label, **properties)
        
def create_or_get_node_smp(tx, label, match_property, properties):
    query = f"MERGE (n:{label} {{{match_property}: $value}}) SET n += $properties RETURN n"
#     result = tx.run(query, value=properties[match_property], properties=properties)
#     if not result.peek():
    create_nodes(tx, label, **properties)
        
def create_relation(tx, node1, node2, relation, property1, property2, label1, label2):
    query = f"MATCH (n1:{label1}), (n2:{label2}) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2)
    count = result.single()[0]
    return count




founding_body_list = founding_body.split(",")    
country_list = country.split(",")
institution_list = institution.split(",")
orcid_list = orcid.split(",")
authors_list_list = authors_list.split(",")

# Create nodes for Researcher, Institution and Country
with driver.session() as session:
    with session.begin_transaction() as tx:
    
        for country_ in country_list:
            if country_ not in ('','-','nan'):
                properties = {'name': country_}
                create_or_get_node(tx, "Country",name_property, properties)
                
        for founding_body_ in founding_body_list:
            if founding_body_ not in ('','-','nan'):
                properties = {'name': founding_body_}
                create_or_get_node(tx, "FundingBody",name_property, properties)
        
    
        for institution_ in institution_list:
            if institution_ not in ('','-','nan'):
                properties = {'name': institution_}
                create_or_get_node(tx, "Institution",name_property, properties)

        for authors_list_,orcid_ in zip(authors_list_list,orcid_list):
            if orcid_ not in ('', '-', 'nan'):
                properties = {'name': authors_list_, 'orcid': orcid_}
                create_or_get_node(tx, "Researcher", 'orcid', properties)

        
#         if orcid not in ('','-','nan'): 
#             create_or_get_node(tx, "Researcher", name=authors_list, orcid=orcid)
        
        properties = {'name': measurementid,'experiment_start':measurement_date,
                    'date_added':upload_date, 'run_title' :experiment_title, 'experiment_id':experimentid, 'access_conditions':access_conditions}
        create_or_get_node(tx, "Measurement",name_property,properties)        
        
        if subcomponent not in ('','-','nan'):    
            subcomponent_uid=subcomponent + '_' + measurementid
            properties = {'name': subcomponent, 'uid':subcomponent_uid}
            create_or_get_node(tx, "Subcomponent",name_property, properties)

        
#         if access_conditions not in ('','-','nan'):    
#             properties = {'name': access_conditions}
#             create_or_get_node(tx, "Parameter",'name', properties)

        
                
        if component not in ('','-','nan'): 
            component_uid=component +"_" + measurementid
            properties = {'name': component, 'uid':component_uid}
            create_or_get_node(tx, "Component",name_property, properties)
  
        if device not in ('','-','nan'): 
            device_uid=device + "_" + measurementid
            properties = {'name': device, 'uid':device_uid}
            create_or_get_node(tx, "Device",name_property, properties)

        

        if material not in ('','-','nan'):  
            material_uid=material + "_" + measurementid
            properties = {'name': material, 'uid':material_uid}
            create_or_get_node(tx, "Material", name_property,properties)

                     

        
        
           
# Create relations between the nodes  
    with session.begin_transaction() as tx:  
    # Check and create relationship
        for orcid_,institution_  in zip(orcid_list,institution_list):
            count = create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', name_property,'Researcher', 'Institution')

            if count == 0:
#                 print("Relationship AFFILIATED_TO doesn't exist, creating it...")
                create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', name_property,'Researcher', 'Institution')
            else:
                pass

        for institution_, country_ in zip(institution_list,country_list):

            count = create_relation(tx, institution_, country_, 'IN', 'name', name_property,'Institution', 'Country')

            if count == 0:
#                 print("Relationship IN doesn't exist, creating it...")
                create_relation(tx, institution_, country_, 'IN', 'name', name_property,'Institution', 'Country')
            else:
                pass
        
        
        for orcid_, country_ in zip(orcid_list,country_list):
            count = create_relation(tx, orcid_, country_, 'IN', 'orcid', name_property,'Researcher', 'Country')

            if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
                create_relation(tx, orcid_, country_, 'IN', 'orcid', name_property,'Researcher', 'Country')
            else:
                pass
        
        
        for orcid_ in orcid_list:
            count = create_relation(tx, measurementid, orcid_, 'BY', name_property, 'orcid','Measurement', 'Researcher')

            if count == 0:
#                 print("Relationship BY doesn't exist, creating it...")
                create_relation(tx, measurementid, orcid_, 'BY', name_property, 'orcid','Measurement', 'Researcher')
            else:
                pass
        
        
        count = create_relation(tx, material_uid, measurementid, 'IS_MEASUREMENT_INPUT', 'uid', name_property,'Material', 'Measurement')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,material_uid, measurementid, 'IS_MEASUREMENT_INPUT', 'uid',name_property,'Material', 'Measurement')
        else:
            pass
        
      
        count = create_relation(tx, component_uid, subcomponent_uid, 'HAS_PART', 'uid', 'uid','Component', 'Subcomponent')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,component_uid, subcomponent_uid, 'HAS_PART', 'uid', 'uid','Component', 'Subcomponent')
        else:
            pass
            
            
        count = create_relation(tx, device_uid, component_uid, 'HAS_PART', 'uid', 'uid','Device', 'Component')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  device_uid, component_uid, 'HAS_PART', 'uid', 'uid','Device', 'Component')
        else:
            pass

        
        count = create_relation(tx, component_uid, material_uid, 'HAS_PART', 'uid', 'uid','Component', 'Material')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  component_uid, material_uid, 'HAS_PART', 'uid', 'uid','Component', 'Material')
        else:
            pass

        count = create_relation(tx, component_uid, measurementid, 'IS_MEASUREMENT_INPUT', 'uid', name_property,'Component', 'Measurement')
        if count == 0:
#                 print("Relationship IN_ orcid doesn't exist, creating it...")
            create_relation(tx,  component_uid, measurementid, 'IS_MEASUREMENT_INPUT', 'uid', name_property,'Component', 'Measurement')
        else:
            pass
            
          
        for founding_body_ in founding_body_list:
            count = create_relation(tx,measurementid , founding_body_,  'FUNDED_BY', name_property, name_property,'Measurement', 'FundingBody')
        if count == 0:
#                 print("Relationship BY doesn't exist, creating it...")
            create_relation(tx, measurementid , founding_body_,  'FUNDED_BY', name_property, name_property,'Measurement', 'FundingBody')
        else:
            pass
        
        


# 5. To get the classes and subclasses from ontology, so as to create further IS_A relations with file data

In [7]:

def get_all_subclasses(driver, target_class_name,node_lab):
    with driver.session() as session:
        
        result = session.run(
            "MATCH (target:{node_lab} {{name: $targetClassName}})<-[:EMMO_IS_A*]-(subclass:{node_lab}) "
            "RETURN DISTINCT subclass.name".format(node_lab=node_lab),
            targetClassName=target_class_name
        )
        return set([record["subclass.name"] for record in result])


In [8]:

target_class_name = "Component"
node_lab=emmo_mat


# Call the function to retrieve all subclasses
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)

# Include the target class itself in the set of subclasses
all_subclasses.add(target_class_name)

# Convert the set to a list
component_lst = list(all_subclasses)

# Print the subclasses
# print(component_lst)

In [9]:
target_class_name = "Device"
node_lab=emmo_mat

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
device_lst = list(all_subclasses)
# print(device_lst)

In [10]:
target_class_name = "Subcomponent"
node_lab=emmo_mat

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
subcomponent_lst = list(all_subclasses)
# print(subcomponent_lst)

In [11]:
target_class_name = "Material"
node_lab=emmo_mat

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
material_lst = list(all_subclasses)
# print(material_lst)

In [12]:
target_class_name = "Manufacturing"
node_lab=emmo_proc

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
manufacturing_lst = list(all_subclasses)
# print(manufacturing_lst)

In [13]:
target_class_name = "Imaging"
node_lab=emmo_proc

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
imaging_lst = list(all_subclasses)
# print(imaging_lst)

In [14]:
target_class_name = "Quantity"
node_lab=emmo_quant

all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
quantity_lst = list(all_subclasses)
# print(quantity_lst)

# 6. Creation of IS_A relationship of meta data (organizational data) to EMMO classes.

In [15]:
from sentence_transformers import SentenceTransformer, util
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 3 most similar sentences in the corpus.
"""

def modify_strings(string):
    modified_lst=''
   

    if len(string.split())==1:
        print('string is ',string)
        if string.isupper()==True:         
            modified_lst=string
            
        elif string[0].isupper()==True and string[1].isupper()==False:
            modified_lst=string
#                 print(modified_string)

        else:
            modified_lst=string.title()

    else:     

        modified_string = ''.join(word.capitalize() for word in string.split())
        modified_lst=modified_string
    return modified_lst



def emmo_cls_onto(queries,emmo_lst,node_label,superclass_name):
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    cls_lst_onto = emmo_lst
    cls_str_onto = []
    for n in cls_lst_onto:
        cls_str_onto.append(str(n))
    #print(abc_str)


    corpus = cls_str_onto
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

    # Query sentences:
    # queries = class_list
    # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(3, len(corpus))

    subclass_name = queries
#         words = subclass_name.split()
#         capitalized_words = [word.capitalize() for word in words]
#         query = ''.join(capitalized_words)
    query = modify_strings(subclass_name)
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    c_lst=[]
    scr_lst=[]

    # We use cosine-similarity and torch.topk to find the highest 3 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
#     print(top_results)
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print('c_lst ',corpus[idx], "(Score: {:.4f})".format(score))
        c_lst.append(corpus[idx].replace('mason.', ''))
        scr_lst.append("{:.4f}".format(score)) 
#         print('c_lst ',c_lst)


    with driver.session() as session:
        with session.begin_transaction() as tx:

            if float(scr_lst[0])>=0.9:
                spr_cls = c_lst[0]  
                count = create_relation(tx,subclass_name ,spr_cls, is_a_rel, name_property, name_property,superclass_name,node_label)
                if count == 0:
                    create_relation(tx,subclass_name ,spr_cls, is_a_rel, name_property,name_property,superclass_name,node_label)
                else:
                    pass
                
                print(query,c_lst[0],superclass_name)

#                 with session.begin_transaction() as tx:   
#                     if device not in ('','-','nan'):  
#                         properties = {'name': material}
#                         create_or_get_node(tx, "Material", properties)


#             elif float(scr_lst[0])<0.6:     
#                 # Example usage

#                 # Create a Neo4j driver instance
#                 # Call the function to create the subclass and "IS_A" relationship
#                 url_mat='http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
#                 create_subclass(driver, node_label, superclass_name, query,relation_emmo_is_a, url_mat)
#                 count = create_relation(tx, subclass_name ,query, "IS_A",  'name', 'name',superclass_name,node_label)
#                 if count == 0:
#                     create_relation(tx, subclass_name ,query, "IS_A",  'name', 'name',superclass_name,node_label)
#                 else:
#                     pass
                

                print('outside func ',superclass_name ,node_label, query)
#            createNewClass(query,spr_cls)

In [16]:
emmo_cls_onto(component,component_lst,emmo_mat,"Component")
emmo_cls_onto(device,device_lst,emmo_mat,"Device")
emmo_cls_onto(subcomponent,subcomponent_lst,emmo_mat,"Subcomponent")
emmo_cls_onto(material,material_lst,emmo_mat,"Material")

string is  WorkingElectrode




Query: WorkingElectrode

Top 3 most similar sentences in corpus:
c_lst  WorkingElectrode (Score: 1.0000)
c_lst  SimpleElectrode (Score: 0.7683)
c_lst  ReferenceElectrode (Score: 0.7392)
WorkingElectrode WorkingElectrode Component
outside func  Component EMMOMatter WorkingElectrode
string is  FuelCell




Query: FuelCell

Top 3 most similar sentences in corpus:
c_lst  FuelCell (Score: 1.0000)
c_lst  FuelCellStack (Score: 0.8851)
c_lst  SingleFuelCell (Score: 0.6559)
FuelCell FuelCell Device
outside func  Device EMMOMatter FuelCell
string is  CatalystLayer




Query: CatalystLayer

Top 3 most similar sentences in corpus:
c_lst  GasDiffusionLayer (Score: 0.4745)
c_lst  TitaniumBasedDiamondLikeCarbonCoatedBipolarPlate (Score: 0.4340)
c_lst  TitaniumBasedGraohiteCoatedBipolarPlate (Score: 0.4150)
string is  LiquidElectrolyte




Query: LiquidElectrolyte

Top 3 most similar sentences in corpus:
c_lst  LiquidElectrolyte (Score: 1.0000)
c_lst  SolidElectrolyte (

In [17]:
# print(Measurement_ID,Experiment_ID=Experiment_ID, Experiment_Title=Experiment_Title, Measurement_date=Measurement_date)

# 7. Get the Characterization Data from file

In [18]:
# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Characterization Data'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Sample preparation'].index[0]

# Extract the organizational data rows
Characterization_measurement_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = Characterization_measurement_data.iloc[:, 0].tolist()
values = Characterization_measurement_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
data_dict = dict(zip(attributes, values))

# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    globals()[var_name] = value

# Print the variables to verify they contain the expected values
for attribute, value in zip(attributes, values):
    var_name = attribute.lower().replace(' ', '_').replace('-', '').replace(':', '')
    print(f"{var_name}: {globals()[var_name]}")


measurement_type: TEMImaging
specimen: homogeneous powder


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_13476\3654495869.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()


In [19]:
def emmo_cls_mapping(queries,emmo_lst,node_label):
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    cls_lst_onto = emmo_lst
    cls_str_onto = []
    for n in cls_lst_onto:
        cls_str_onto.append(str(n))
    #print(abc_str)


    corpus = cls_str_onto
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

    # Query sentences:
    # queries = class_list
    # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(3, len(corpus))

    subclass_name = queries
#         words = subclass_name.split()
#         capitalized_words = [word.capitalize() for word in words]
#         query = ''.join(capitalized_words)
    query = subclass_name
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    c_lst=[]
    scr_lst=[]

    # We use cosine-similarity and torch.topk to find the highest 3 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
#     print(top_results)
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print('c_lst ',corpus[idx], "(Score: {:.4f})".format(score))
        c_lst.append(corpus[idx].replace('mason.', ''))
        scr_lst.append("{:.4f}".format(score)) 
#         print('c_lst ',c_lst)


    with driver.session() as session:
        with session.begin_transaction() as tx:

            if float(scr_lst[0])>=0.9:
                spr_cls = c_lst[0]  
                return spr_cls
    


In [20]:
uid_measurement_type=measurement_type+'_'+measurementid

with driver.session() as session:
    with session.begin_transaction() as tx:
        if measurement_type not in ('','-','nan'):    
            properties = {'name': measurement_type,'uid' : uid_measurement_type}
            create_or_get_node(tx, "MeasurementType",name_property, properties)


        count = create_relation(tx, uid_measurement_type, measurement_type, is_a_rel, 'uid', name_property,'MeasurementType', emmo_proc)

        if count == 0:
        #                 print("Relationship IN doesn't exist, creating it...")
            create_relation(tx, uid_measurement_type, measurement_type, is_a_rel, 'uid', name_property,'MeasurementType', emmo_proc)
        else:
            pass
        
        
        count = create_relation(tx, measurementid, uid_measurement_type, 'HAS_MEASUREMENT_TYPE', name_property, 'uid','Measurement', 'MeasurementType')

        if count == 0:
        #                 print("Relationship IN doesn't exist, creating it...")
            create_relation(tx, measurementid, uid_measurement_type, 'HAS_MEASUREMENT_TYPE', name_property, 'uid','Measurement', 'MeasurementType')
        else:
            pass

## Get Sample preparation from Characterization environment

In [21]:
# Find the row index where 'Organizational Data' is located

# Find the row index where 'Sample preparation' is located
# Find the row index where 'Sample preparation' is located
start_row = 'Sample preparation'
end_row = 'Characterization environment'

start_index = df[df.iloc[:, 0] == start_row].index[0]
end_index = df[df.iloc[:, 0] == end_row].index[0]

# Extract the desired table based on the row indices
table_data = df.iloc[start_index+1:end_index]

# Assign the first row as column names
table_data.columns = table_data.iloc[0]
table_data = table_data.iloc[1:].reset_index(drop=True)

# Remove the unwanted numerical column
table_data = table_data.loc[:, ~table_data.columns.astype(str).str.isnumeric()]
table_data


23,Step,Precursor,Amount,Technique,Condition,Target,Amount,NaN
0,1,CarbonBlack,12.5 mm,SamplePreparation,"Duration:24 hour, Temperature: 30 °C",PTFESubstrate,5 mm,NaN
1,2,"F30E, HomoGenousMaterial","5 mm, 2.3 mm",CCLManufacturing,"Temperature: 50 °C, Duration: 10 min, Electric...",Polyepoxide,6 mm,NaN
2,3,"Polyepoxide, PTFESubstrate, FurnaceBlack","6 mm, 5 mm, 3 mm",5LayerMEAManufacturing,Duration:24 hour,LiquidElectrolyte,3 mm,NaN


In [22]:
# Loop through each row and print the column values
for index, row in table_data.iterrows():
    print(f"Row {index+1}:")
    for column, value in row.items():
        if not pd.isna(value):
            print(f"{column}: {value}")
    print()

Row 1:
Step: 1
Precursor: CarbonBlack
Amount: 12.5 mm
Technique: SamplePreparation
Condition: Duration:24 hour, Temperature: 30 °C
Target: PTFESubstrate
Amount: 5 mm

Row 2:
Step: 2
Precursor: F30E, HomoGenousMaterial
Amount: 5 mm, 2.3 mm
Technique: CCLManufacturing
Condition: Temperature: 50 °C, Duration: 10 min, ElectricCurrent:250 pA, Energy:5 keV
Target: Polyepoxide
Amount: 6 mm

Row 3:
Step: 3
Precursor: Polyepoxide, PTFESubstrate, FurnaceBlack
Amount: 6 mm, 5 mm, 3 mm
Technique: 5LayerMEAManufacturing
Condition: Duration:24 hour
Target: LiquidElectrolyte
Amount: 3 mm



In [23]:
sample_name = "SamplePreparation_" + measurementid
with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': sample_name}
        create_or_get_node(tx, "SamplePreparation",name_property, properties)
        create_relation(tx, measurementid, sample_name, 'HAS_SAMPLEPREPARATION', name_property, name_property, 'Measurement', 'SamplePreparation')

# 8. get materials again-

In [24]:
node_label_mat = emmo_mat
node_label_tech =emmo_proc
node_label_quantity =emmo_quant
superclass_step='SamplePreparation'
superclass_fabrication_step='FabricationWorkflow'

superclass_preprocess_analyse='DataPreprocessAnalysis'

superclass_char_env='CharacterizationEnvironment'
superclass_tech_data='TechnicalData'
superclass_equipment_data='EquipmentSetup'
sample_name = "SamplePreparation_" + measurementid
fabrication_name = "FabricationWorkflow_" + measurementid
preporcess_analysis_name = "DataPreprocessAnalysis_" + measurementid

superclass_mat = 'Material'
superclass_comp = 'Component'
superclass_subcomp = 'Subcomponent'
superclass_device = 'Device'
superclass_tech='Manufacturing'
superclass_para='Parameter'
has_float_property='HAS_FLOAT_VALUE'
has_manufacturing_input='HAS_MANUFACTURING_INPUT'
is_manufacturing_input='IS_MANUFACTURING_INPUT'
is_manufacturing_output='IS_MANUFACTURING_OUTPUT'
has_parameter_rel='HAS_PARAMETER'


In [25]:
def create_rel_property(tx, node1, node2, relation, property1, property2, label1, label2, properties):
    query = f"MATCH (n1:{label1}), (n2:{label2}) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"SET r += $properties " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2, properties=properties)
    count = result.single()[0]
    return count


In [26]:


def extract_numeric_value(item):
    pattern = r'(\d+(\.\d+)?)'  # Regex pattern to match numeric values
    
    matches = re.findall(pattern, item)
    if matches:
        numeric_value = float(matches[0][0])
        return numeric_value
    else:
        return None
    



def extract_parameter_names(item):
    pattern = r'(\w+):'  # Regex pattern to match variable names

    variable_names = re.findall(pattern, item)
    
    if len(variable_names) >= 1:
        return variable_names[0]
    else:
        return item

In [27]:



def generate_random_id():
    # Get the current date and time
    current_datetime = datetime.now()
    
    # Format the date and time as desired (e.g., YYYYMMDD_HHMMSS)
    datetime_string = current_datetime.strftime("%Y%m%d_%H%M%S")
    
    # Generate a random number or string
    random_part = str(random.randint(1000, 999999))
    
    # Combine the date, time, and random part to create the ID
    random_id = datetime_string + random_part
    
    return random_id


In [28]:
generate_random_id()

'20230904_111602709752'

In [29]:
# Define the column-to-variable mapping
def char_steps(node_label_name,superclass_name):
    column_mapping = {
        'Step': 'step',
        'Precursor': 'precursor',
        'Amount': 'amount',
        'Technique': 'technique',
        'Condition': 'condition',
        'Target': 'target'
    }

    # Define variables
    
    previous_node=superclass_name
    uid_prop_name='uid'
    previous_targate={}

    # Loop through each row and store the column values dynamically
    for index, row in table_data.iterrows():
        variables = {variable: '' for variable in column_mapping.values()}
        print(f"Row {index+1}:")
        amounts = []
        for column, value in row.items():
            if not pd.isna(value):
                if column == 'Amount':
                    amounts.append(value)
                else:
                    variable_name = column_mapping.get(column)
                    if variable_name:
                        variables[variable_name] = value

        # Store the amounts separately
        amount1, amount2 = amounts[:2]
        print(amount1)
        # Access the values from the variables dictionary
        step = variables['step']
        step_name='Step_'+str(step)+'_'+measurementid
        precursor = variables['precursor']
        technique = variables['technique']
        condition = variables['condition']
        target = variables['target']

        precursor_list = precursor.split(', ')
        amount1_list= amount1.split(', ')

        with driver.session() as session:
            with session.begin_transaction() as tx:

                if technique not in ('','-','nan'):
                    tech_prop_name=uid_prop_name
                    tech_unique_id=technique+'_'+generate_random_id()
                    properties = {name_property: technique, tech_prop_name: tech_unique_id}
                    create_or_get_node_smp(tx,superclass_tech,name_property, properties)



        print(precursor_list)        
        superclass_val=''                

    #     for prec in precursor_list:
        for prec, amt in zip(precursor_list, amount1_list):
            print('This is the prec: ', prec, ' This is the amount: ', amt)

            with driver.session() as session:
                with session.begin_transaction() as tx:
    #                 superclass_val = superclass_mat
                    prec_val = emmo_cls_mapping(prec, material_lst, node_label_mat)

                    if prec_val is not None:
                        prec_prop_name=uid_prop_name
                        prec_unique_id=prec_val+'_'+generate_random_id()
                        properties = {name_property: prec, prec_prop_name: prec_unique_id }
                        if prec_val in previous_targate:
                            pass
                        else:
                            create_or_get_node_smp(tx, node_label_name, name_property, properties)
                        create_relation(tx,prec ,prec_val, is_a_rel, name_property, name_property,node_label_name,node_label_mat)
#                         print(prec_val, ' its here ', superclass_val)
                    else:
    #                     superclass_val = superclass_comp
                        prec_val = emmo_cls_mapping(prec, component_lst, node_label_mat)

                        if prec_val is not None:
                            prec_prop_name=uid_prop_name
                            prec_unique_id=prec_val+'_'+generate_random_id()
                            properties = {name_property: prec, prec_prop_name: prec_unique_id }
                            if prec_val in previous_targate:
                                pass
                            else:
                                create_or_get_node_smp(tx, node_label_name, name_property, properties)
                            create_relation(tx,prec ,prec_val, is_a_rel,name_property, name_property,node_label_name,node_label_mat)
#                             print(prec_val, ' its here ', superclass_val)
                        else:
    #                         superclass_val = superclass_device
                            prec_val = emmo_cls_mapping(prec, device_lst, node_label_mat)

                            if prec_val is not None:
                                prec_prop_name=uid_prop_name
                                prec_unique_id=prec_val+'_'+generate_random_id()
                                properties = {name_property: prec, prec_prop_name: prec_unique_id}
                                if prec_val in previous_targate:
                                    pass
                                else:
                                    create_or_get_node_smp(tx, node_label_name, name_property, properties)
                                create_relation(tx,prec ,prec_val, is_a_rel, name_property,name_property,node_label_name,node_label_mat)
#                                 print(prec_val,' ',prec, ' its here ', superclass_val)
                            else:
    #                             superclass_val = superclass_subcomp
                                prec_val = emmo_cls_mapping(prec, subcomponent_lst, node_label_mat)

                                if prec_val is not None:
                                    prec_prop_name=uid_prop_name
                                    prec_unique_id=prec_val+'_'+generate_random_id()
                                    properties = {name_property: prec, prec_prop_name: prec_unique_id}
                                    if prec_val in previous_targate:
                                        pass
                                    else:
                                        create_or_get_node_smp(tx, node_label_name, name_property, properties)
                                    create_relation(tx,prec ,prec_val, is_a_rel,name_property, name_property,node_label_name,node_label_mat)
#                                     print(prec_val, ' its here ', superclass_val)
                                else:
                                    pass

                    create_relation(tx, previous_node , prec_unique_id, has_manufacturing_input, name_property, prec_prop_name,node_label_name,node_label_name)
                    numeric_amt1=extract_numeric_value(amt)
                    properties = {
                        has_float_property: numeric_amt1
                    }

                    if prec_val in previous_targate:
                        create_rel_property(tx, previous_targate[prec_val] , tech_unique_id, is_manufacturing_input, prec_prop_name, tech_prop_name,node_label_name,superclass_tech,properties)
                    else:
                        print("Value is not present in the list")
                        create_rel_property(tx, prec_unique_id , tech_unique_id, is_manufacturing_input,  prec_prop_name, tech_prop_name,node_label_name,superclass_tech,properties)



        tech_manuf=emmo_cls_mapping(technique,manufacturing_lst,node_label_tech)   

#         superclass_val = superclass_mat
        tar_var = emmo_cls_mapping(target, material_lst, node_label_mat)
        with driver.session() as session:
            with session.begin_transaction() as tx:
                if tar_var is not None:

                    tar_prop_name=uid_prop_name
                    tar_unique_id=tar_var+'_'+generate_random_id()
                    properties = {name_property: target, tar_prop_name: tar_unique_id}
                    create_or_get_node_smp(tx, node_label_name,name_property, properties)
                    create_relation(tx,target ,tar_var, is_a_rel,name_property, name_property,node_label_name,node_label_mat)
                else:
#                     superclass_val = superclass_comp
                    tar_var = emmo_cls_mapping(target, component_lst, node_label_mat)

                    if tar_var is not None:
                        tar_prop_name=uid_prop_name
                        tar_unique_id=tar_var+'_'+generate_random_id()
                        properties = {name_property: target, tar_prop_name: tar_unique_id}
                        create_or_get_node_smp(tx, node_label_name, name_property, properties)
                        create_relation(tx,target ,tar_var, is_a_rel, name_property, name_property,node_label_name,node_label_mat)
                    else:
#                         superclass_val = superclass_device
                        tar_var = emmo_cls_mapping(target, device_lst, node_label_mat)

                        if tar_var is not None:
                            tar_prop_name=uid_prop_name
                            tar_unique_id=tar_var+'_'+generate_random_id()
                            properties = {name_property: target, tar_prop_name: tar_unique_id}
                            create_or_get_node_smp(tx, node_label_name, name_property, properties)
                            create_relation(tx,target ,tar_var,is_a_rel, name_property, name_property,node_label_name,node_label_mat)
                        else:
#                             superclass_val = superclass_subcomp
                            tar_var = emmo_cls_mapping(target, subcomponent_lst, node_label_mat)
                            if tar_var is not None:
                                tar_prop_name=uid_prop_name
                                tar_unique_id=tar_var+'_'+generate_random_id()
                                properties = {name_property: target, tar_prop_name: tar_unique_id}
                                create_or_get_node_smp(tx, node_label_name, name_property, properties)
                                create_relation(tx,target ,tar_var, is_a_rel, name_property, name_property,node_label_name,node_label_mat)
                            else:
                                pass
                numeric_amt2=extract_numeric_value(amount2)
                properties = {
                        has_float_property: numeric_amt2
                    }
                create_rel_property(tx, tech_unique_id ,tar_unique_id , is_manufacturing_output, tech_prop_name ,tar_prop_name,superclass_tech,node_label_name,properties )
                create_relation(tx,tech_unique_id ,tech_manuf, is_a_rel, tech_prop_name, name_property,superclass_tech,node_label_tech)

        previous_targate[tar_var]=(tar_unique_id)

        condition_list = condition.split(', ') 

        for condition_ in condition_list:
            with driver.session() as session:
                with session.begin_transaction() as tx:



                    if condition_ not in ('','-','nan'):
                        condition=extract_parameter_names(condition_)
                        condition_var = emmo_cls_mapping(condition, quantity_lst, node_label_quantity)
                        print(condition_var)
                        condition_prop_name=uid_prop_name
                        condition_unique_id=condition+"_"+generate_random_id()
                        properties = {name_property: condition,'measurementid':measurementid, 'stepid':step, condition_prop_name:  condition_unique_id}
                        create_or_get_node_smp(tx, superclass_para,condition_prop_name, properties)       
                        numeric_condition=extract_numeric_value(condition_)
                        properties = {has_float_property: numeric_condition}
                        create_relation(tx, condition, condition_var, is_a_rel, name_property, name_property,superclass_para,node_label_quantity)
                        create_rel_property(tx, tech_unique_id, condition_unique_id, has_parameter_rel, tech_prop_name, condition_prop_name ,superclass_tech,superclass_para, properties)
        previous_node= '' 




    print(previous_targate)

                    
char_steps(superclass_step,sample_name)

Row 1:
12.5 mm
['CarbonBlack']
This is the prec:  CarbonBlack  This is the amount:  12.5 mm




Query: CarbonBlack

Top 3 most similar sentences in corpus:
c_lst  CarbonBlack (Score: 1.0000)
c_lst  AcetyleneBlack (Score: 0.6581)
c_lst  ThermalBlack (Score: 0.6580)
Value is not present in the list




Query: SamplePreparation

Top 3 most similar sentences in corpus:
c_lst  SamplePreparation (Score: 1.0000)
c_lst  CCMManufacturing (Score: 0.1520)
c_lst  CCLManufacturing (Score: 0.1261)




Query: PTFESubstrate

Top 3 most similar sentences in corpus:
c_lst  PTFESubstrate (Score: 1.0000)
c_lst  PTFE (Score: 0.7922)
c_lst  PlatinumCatalyst (Score: 0.4594)




Query: Duration

Top 3 most similar sentences in corpus:
c_lst  Duration (Score: 1.0000)
c_lst  Length (Score: 0.6758)
c_lst  TimeInterval (Score: 0.4974)
Duration




Query: Temperature

Top 3 most similar sentences in corpus:
c_lst  Temperature (Score: 1.0000)
c_lst  Heat (Score: 0.5619)
c_lst  ThermalInsulance (Score: 0.5557)
Tempe

# 9. Get data for Characterization environment

In [30]:
### conect it to measurementid

char_env_name = superclass_char_env+"_" + measurementid
with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': char_env_name}
        create_or_get_node(tx, superclass_char_env,name_property, properties)
       

In [31]:
# new = emmo_cls_mapping(target, subcomponent_lst, node_label_mat)

In [32]:
# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Characterization environment'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Equipment setup'].index[0]

# Extract the Characterization environment data rows
Characterization_measurement_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = Characterization_measurement_data.iloc[:, 0].tolist()
values = Characterization_measurement_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
char_env_data_dict = {}
uid_prop_name='uid'


# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    key_parts = attribute.split('/')
    var_name = key_parts[0].strip().lower().replace(' ', '_').replace('-', '').replace(':', '')
    char_env_data_dict[var_name] = value

# Print the variables to verify they contain the expected values
for key, value in char_env_data_dict.items():
    print(f"{key}: {value}")
    key_val=emmo_cls_mapping(key, quantity_lst, node_label_quantity)
    print(key_val)
    if key_val is not None:
        env_prop_name=uid_prop_name
        env_unique_id= key +"_"+ generate_random_id()
        with driver.session() as session:
            with session.begin_transaction() as tx:

                properties = {name_property: key_val,'measurementid':measurementid, env_prop_name:  env_unique_id}
                create_or_get_node_smp(tx, superclass_para,env_prop_name, properties)     

    #             numeric_condition=extract_numeric_value(condition_)
                properties = {has_float_property: value}
                print(key_val, key_val,superclass_char_env,node_label_quantity)
                create_relation(tx, key, key_val, is_a_rel, name_property, name_property,superclass_char_env,node_label_quantity)
                create_rel_property(tx, char_env_name, env_unique_id, has_parameter_rel, name_property,  env_prop_name,superclass_char_env,superclass_para, properties)

temperature: 23


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_13476\1484524854.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()






Query: temperature

Top 3 most similar sentences in corpus:
c_lst  Temperature (Score: 1.0000)
c_lst  Heat (Score: 0.5619)
c_lst  ThermalInsulance (Score: 0.5557)
Temperature
Temperature Temperature CharacterizationEnvironment EMMOQuantity
humidity: 50




Query: humidity

Top 3 most similar sentences in corpus:
c_lst  DryingTemperature (Score: 0.4794)
c_lst  Temperature (Score: 0.4509)
c_lst  MassConcentrationOfWaterVapour (Score: 0.4230)
None
atmosphere: air




Query: atmosphere

Top 3 most similar sentences in corpus:
c_lst  Pressure (Score: 0.5093)
c_lst  Heat (Score: 0.4098)
c_lst  Energy (Score: 0.3877)
None
pressure: 1




Query: pressure

Top 3 most similar sentences in corpus:
c_lst  Pressure (Score: 1.0000)
c_lst  PressureCoefficient (Score: 0.6492)
c_lst  Stress (Score: 0.5593)
Pressure
Pressure Pressure CharacterizationEnvironment EMMOQuantity


# 10. Get data for Tech Data

In [33]:
# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Technical Data'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Synthesis/Fabrication Workflow'].index[0]

# Extract the Characterization environment data rows
Characterization_measurement_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = Characterization_measurement_data.iloc[:, 0].tolist()
values = Characterization_measurement_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
tech_data_dict = {}



# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    key_parts = attribute.split('/')
    var_name = key_parts[0].strip().lower().replace(' ', '_').replace('-', '').replace(':', '')
    tech_data_dict[var_name] = value

# Print the variables to verify they contain the expected values
for key, value in tech_data_dict.items():
    print(f"{key}: {value}")


format: tiff
file_size: 1
file_name: Test.tif
data_type: 8 bit Gray
dimension_x: 1024
dimension_y: 1024
scale: 8.1
mask: yes
mask_link: link


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_13476\3752677002.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Characterization_measurement_data.loc[:, 0] = Characterization_measurement_data.loc[:, 0].str.strip()


In [34]:
tech_data_name = superclass_tech_data+"_" + measurementid
has_technicaldata_rel='HAS_TECHNICALDATA'
with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': tech_data_name}
        properties.update(tech_data_dict)
        create_or_get_node(tx, superclass_tech_data,name_property, properties)
        create_relation(tx, measurementid, tech_data_name, has_technicaldata_rel, name_property, name_property, 'Measurement', superclass_tech_data)
       

In [35]:
properties.update(tech_data_dict)

In [36]:
properties

{'name': 'TechnicalData_000000-1-2',
 'format': 'tiff',
 'file_size': 1,
 'file_name': 'Test.tif',
 'data_type': '8 bit Gray',
 'dimension_x': '1024',
 'dimension_y': '1024',
 'scale': '8.1',
 'mask': 'yes',
 'mask_link': 'link'}

# 11. Get data for Equipment Setup Data


In [37]:
# Find the row index where 'Organizational Data' is located
start_row = df[df.iloc[:, 0] == 'Equipment setup'].index[0]

# Find the row index where 'Characterization Data' is located
end_row = df[df.iloc[:, 0] == 'Calibration'].index[0]

# Extract the Characterization environment data rows
equipment_data = df.iloc[start_row + 1: end_row]

# Remove trailing spaces from attribute names
equipment_data.loc[:, 0] = equipment_data.loc[:, 0].str.strip()

# Extract attributes and values
attributes = equipment_data.iloc[:, 0].tolist()
values = equipment_data.iloc[:, 1].tolist()

# Create a dictionary of attributes and values
equipment_data_dict = {}



# Generate variable names dynamically and assign values
for attribute, value in zip(attributes, values):
    key_parts = attribute.split('/')
    var_name = key_parts[0].strip().lower().replace(' ', '_').replace('-', '').replace(':', '')
    equipment_data_dict[var_name] = value

# Print the variables to verify they contain the expected values
for key, value in equipment_data_dict.items():
    print(f"{key}: {value}")


microscope: Zeiss Gemini Ultra plus
acceleration_voltage: 20
magnification: 250
cathode: LaB6
working_distance: 8.5
probe: Electron beam
detector: InLens
imaging_technique: nan
signal: Secondary electrons
time_lapse: 30
raw_data: electron images
data_adquisition_rate: -


C:\Users\a.mathur\AppData\Local\Temp\ipykernel_13476\144186454.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  equipment_data.loc[:, 0] = equipment_data.loc[:, 0].str.strip()


In [38]:
equipment_setup_name = superclass_equipment_data+"_" + measurementid
has_equipmentsetup_rel='HAS_EQUIPMENTSETUP'
with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': equipment_setup_name}
        properties.update(equipment_data_dict)
        create_or_get_node(tx, superclass_equipment_data,name_property, properties)
        create_relation(tx, measurementid, equipment_setup_name, has_equipmentsetup_rel, name_property, name_property, 'Measurement', superclass_equipment_data)
       

# 12. Get  Fabrication environment

In [39]:

start_row = 'Synthesis/Fabrication Workflow'
end_row = 'Performed experiments'

start_index = df[df.iloc[:, 0] == start_row].index[0]
end_index = df[df.iloc[:, 0] == end_row].index[0]

# Extract the desired table based on the row indices
table_data = df.iloc[start_index+1:end_index]

# Assign the first row as column names
table_data.columns = table_data.iloc[0]
table_data = table_data.iloc[1:].reset_index(drop=True)

# Remove the unwanted numerical column
table_data = table_data.loc[:, ~table_data.columns.astype(str).str.isnumeric()]
table_data


64,Step,Precursor,Amount,Technique,Condition,Target,Amount,NaN
0,1,"WorkingElectrode, AcidicElectrochemicalCell, C...","34 mm, 24 mm, 87mm",SamplePreparation,"Duration:24 hour, Temperature: 20 °C",F30E,5 mm,NaN
1,2,"F30E, ElectricCurrentCollector","16mm, 30 mm",FuelCellManufacturing,Temperature:12 mm,CarbonBlack,7 mm,NaN


In [40]:

with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': fabrication_name}
        create_or_get_node(tx, "FabricationWorkflow",name_property, properties)
        create_relation(tx, measurementid, fabrication_name, 'HAS_FABRICATION_WORKFLOW', name_property, name_property, 'Measurement', 'FabricationWorkflow')

In [41]:
char_steps(superclass_fabrication_step,fabrication_name) #need to have amount

Row 1:
34 mm, 24 mm, 87mm
['WorkingElectrode', 'AcidicElectrochemicalCell', 'Cathode']
This is the prec:  WorkingElectrode  This is the amount:  34 mm




Query: WorkingElectrode

Top 3 most similar sentences in corpus:
c_lst  ActiveMaterial (Score: 0.3826)
c_lst  FunctionalMaterial (Score: 0.3433)
c_lst  ManufacturedMaterial (Score: 0.3277)




Query: WorkingElectrode

Top 3 most similar sentences in corpus:
c_lst  WorkingElectrode (Score: 1.0000)
c_lst  SimpleElectrode (Score: 0.7683)
c_lst  ReferenceElectrode (Score: 0.7392)
Value is not present in the list
This is the prec:  AcidicElectrochemicalCell  This is the amount:  24 mm




Query: AcidicElectrochemicalCell

Top 3 most similar sentences in corpus:
c_lst  AcidicElectrolyte (Score: 0.7669)
c_lst  AlkalineElectrolyte (Score: 0.5248)
c_lst  LiquidElectrolyte (Score: 0.4457)




Query: AcidicElectrochemicalCell

Top 3 most similar sentences in corpus:
c_lst  ElectrochemicalComponent (Score: 0.4734)
c_lst  FuelCellComponent (Score

# 13. Get  Main data preprocessing

In [42]:


with driver.session() as session:
    with session.begin_transaction() as tx:
        properties = {'name': preporcess_analysis_name}
        create_or_get_node(tx, superclass_preprocess_analyse,name_property, properties)
        create_relation(tx, measurementid, preporcess_analysis_name, 'HAS_FABRICATION_WORKFLOW', name_property, name_property, 'Measurement', superclass_preprocess_analyse)

In [43]:

def create_df(start_row,end_row):


    start_index = df[df.iloc[:, 0] == start_row].index[0]
    end_index = df[df.iloc[:, 0] == end_row].index[0]

    # Extract the desired table based on the row indices
    table_data = df.iloc[start_index+1:end_index]

    # Assign the first row as column names
    table_data.columns = table_data.iloc[0]
    table_data = table_data.iloc[1:].reset_index(drop=True)

    # Remove the unwanted numerical column
    table_data = table_data.loc[:, ~table_data.columns.astype(str).str.isnumeric()]
    return table_data



start_row = 'Main data preprocessing'
end_row = 'Main data analysis procedure'
df1=create_df(start_row,end_row)

In [44]:

start_row = 'Main data analysis procedure'
end_row = 'Technical Data'
df2=create_df(start_row,end_row)

In [45]:
# Concatenate the dataframes vertically
merged_df = pd.concat([df1, df2], ignore_index=True)

print(merged_df)

  Step             Precursor Amount                    Technique  Condition  \
0    1             Raw image    NaN          Contrast adjustment  factor: 1   
1    2                  Int1    NaN        Brightness adjustment   factor:1   
2    1  Post-processed image    NaN  Manual particle measurement        NaN   

  Software/ algorithm                Target  Amount  
0                 NaN                  Int1     NaN  
1                 NaN  Post-processed image     NaN  
2    Software: ImageJ          Average size  110 um  


In [46]:

def extract_parameters(item):
    parameters = item.split(',')
    param_dict = {}
    for parameter in parameters:
        parts = parameter.split(':')
        if len(parts) == 2:
            param_name = parts[0].strip()
            param_value = parts[1].strip()
            if param_name in param_dict:
                param_dict[param_name].append(param_value)
            else:
                param_dict[param_name] = [param_value]
    return param_dict


In [47]:
# Define the column-to-variable mapping
def char_prep_analysis_steps(node_label_name,superclass_name):
    column_mapping = {
        'Step': 'step',
        'Precursor': 'precursor',
        'Technique': 'technique',
        'Condition': 'condition',
        'Software/ algorithm': 'sw_algo',
        'Target': 'target'
    }

    # Define variables
    previous_node=superclass_name
    uid_prop_name='uid'
    previous_targate={}



    for index, row in merged_df.iterrows():
        variables = {variable: '' for variable in column_mapping.values()}
        print(row,' is row')
        print(f"Row {index+1}:")
        for column, value in row.items():
            if not pd.isna(value):
                if column == 'Amount':
                    pass
                else:
                    variable_name = column_mapping.get(column)
                    if variable_name:
                        variables[variable_name] = value
    

        # Access the values from the variables dictionary
        step = variables['step']
        step_name='Step_'+str(step)+'_'+measurementid
        precursor = variables['precursor']
        technique = variables['technique']
        condition = variables['condition']
        sw_algo = variables['sw_algo']
        target = variables['target']

        precursor_list = precursor.split(', ')

        ###############################################################

        tech_prop_name = uid_prop_name
        tech_unique_id = technique + '_' + generate_random_id()
        software_algorithm_properties = {}  # Dictionary to store software/algorithm attributes

        # Process the sw_algo_list
        sw_algo_list = sw_algo.split(', ')
        for sw_algo_ in sw_algo_list:
            if sw_algo_ not in ('', '-', 'nan'):
                print(sw_algo_, 'sw_algo_')
                param_dict = extract_parameters(sw_algo_)
                for param_name, param_values in param_dict.items():
                    if param_name in software_algorithm_properties:
                        software_algorithm_properties[param_name].extend(param_values)
                    else:
                        software_algorithm_properties[param_name] = param_values

        # Create technique node
        with driver.session() as session:
            with session.begin_transaction() as tx:
                properties = {
                    name_property: technique,
                    tech_prop_name: tech_unique_id,
                }
                for param_name, param_values in software_algorithm_properties.items():
                    properties[param_name] = ', '.join(param_values)  # Join the values as a comma-separated string

                create_or_get_node_smp(tx, node_label_name, name_property, properties)
                    
      ##########################################################################################################              

        print(precursor_list)        
        superclass_val=''                

    #     for prec in precursor_list:
        for prec in (precursor_list):
            print('This is the prec: ', prec)

            with driver.session() as session:
                with session.begin_transaction() as tx:
    #                 superclass_val = superclass_mat
                    prec_val = prec

                    if prec_val is not None:
                        prec_prop_name=uid_prop_name
                        prec_unique_id=prec_val+'_'+generate_random_id()
                        properties = {name_property: prec, prec_prop_name: prec_unique_id }
                        if prec_val in previous_targate:
                            pass
                        else:
                            create_or_get_node_smp(tx, node_label_name, name_property, properties)
                    


                    create_relation(tx, previous_node , prec_unique_id, has_manufacturing_input, name_property, prec_prop_name,node_label_name,node_label_name)
                    print('prec_val in previous_targate',prec_val , previous_targate) 
                    if prec_val in previous_targate:
                        
                        create_relation(tx, previous_targate[prec_val] , tech_unique_id, is_manufacturing_input, prec_prop_name, tech_prop_name,node_label_name,node_label_name)
                    else:
                        print('prec_unique_id , tech_unique_id',prec_unique_id , tech_unique_id) 
                        create_relation(tx, prec_unique_id , tech_unique_id, is_manufacturing_input,  prec_prop_name, tech_prop_name,node_label_name,node_label_name)

                
             



        tech_manuf=technique

#         superclass_val = superclass_mat
        tar_var = target
        with driver.session() as session:
            with session.begin_transaction() as tx:
                if tar_var is not None:

                    tar_prop_name=uid_prop_name
                    tar_unique_id=tar_var+'_'+generate_random_id()
                    properties = {name_property: target, tar_prop_name: tar_unique_id}
                    create_or_get_node_smp(tx, node_label_name,name_property, properties)


                create_relation(tx, tech_unique_id ,tar_unique_id , is_manufacturing_output, tech_prop_name ,tar_prop_name,node_label_name,node_label_name )
        

        previous_targate[tar_var]=(tar_unique_id)

        
        condition_list = condition.split(', ') 
        
        for condition_ in condition_list:
            with driver.session() as session:
                with session.begin_transaction() as tx:

                    if condition_ not in ('','-','nan'):
                        print(condition_,'condition_')
                        condition=extract_parameter_names(condition_)
                        condition_var = condition
                        print(condition_var)
                        condition_prop_name=uid_prop_name
                        condition_unique_id=condition+"_"+generate_random_id()
                        properties = {name_property: condition,'measurementid':measurementid, condition_prop_name:  condition_unique_id}
                        create_or_get_node_smp(tx, node_label_name,condition_prop_name, properties)
                        print(extract_numeric_value(condition_))
                        numeric_condition=extract_numeric_value(condition_)
                        
                        if  not numeric_condition:
                            create_relation(tx, tech_unique_id, condition_unique_id, has_parameter_rel, tech_prop_name, condition_prop_name ,node_label_name,node_label_name)
                            
                        else:
                            properties = {has_float_property: numeric_condition}
                            print('not empty')
                            create_rel_property(tx, tech_unique_id, condition_unique_id, has_parameter_rel, tech_prop_name, condition_prop_name ,node_label_name,node_label_name, properties)
                            print('empty')
           
                            
        previous_node= '' 


    #     print(step, precursor, mat, amount1, technique, condition, target, amount2)

    print(previous_targate)
  

char_prep_analysis_steps(superclass_preprocess_analyse,preporcess_analysis_name)

Step                                     1
Precursor                        Raw image
Amount                                 NaN
Technique              Contrast adjustment
Condition                        factor: 1
Software/ algorithm                    NaN
Target                                Int1
Amount                                 NaN
Name: 0, dtype: object  is row
Row 1:
['Raw image']
This is the prec:  Raw image
prec_val in previous_targate Raw image {}
prec_unique_id , tech_unique_id Raw image_20230904_111711313060 Contrast adjustment_20230904_111711842038
factor: 1 condition_
factor
1.0
not empty
empty
Step                                       2
Precursor                               Int1
Amount                                   NaN
Technique              Brightness adjustment
Condition                           factor:1
Software/ algorithm                      NaN
Target                  Post-processed image
Amount                                   NaN
Name: 1, dtype: obj

In [48]:
# Neo4j codes...TO DELETE DB from Neo4j, in case...

# Match(n) optional match(n)-[r]-() delete n,r

#to search for a node
# MATCH (n:Person)
# WHERE n.name = 'John'
# RETURN n


# MATCH (superclass:EMMOMatter)
# WHERE superclass.name = 'Device'
#     return superclass